In [1]:
import requests
from bs4 import BeautifulSoup
import time
import smtplib
import datetime

In [2]:
url = 'https://www.amazon.in/Redmi-A4-5G-Sparkle-Charging/dp/B0DLW427YG/ref=sr_1_1_sspa?_encoding=UTF8&content-id=amzn1.sym.bd428427-e2fa-455b-845a-a54edb5f536f&dib=eyJ2IjoiMSJ9.Zu16M0f4gcJTITvmFzGqs09EXESHBn36UX9vRwLtp43PhS-3_IY2Fs73Ku6LbRlMIJ-BGLDG7ZZd2ctqJbqxv7vk6mrTF2l87iQWhN2qtRYWXCRutAfVXjCa6s9k8lYTdcgifhH_faDWuAyM1EmFgkLcz5HCH5PoTuKiJkZjPjGH1xy1OtH05DxdSO3P-dJ1heNXxYWzobMdKRP3M5E4Ta0jb0TVF9ZnoKSzGwgWY6u7GVXjITUOJHg7UoPtX3GxEtJrG6wAvgL4S38ajygckC1WHO4hX_D2y3HIHFLe8Lc.L5Y8AffFc-O4ioP4KMYOvwBicFzKvGvZrsYAoA_GYXU&dib_tag=se&pd_rd_r=56178e86-1b26-4ccb-a9fe-28a25be7649f&pd_rd_w=A11pD&pd_rd_wg=MYv29&qid=1746590515&refinements=p_36%3A-1030000&rnid=1318502031&s=electronics&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&th=1'

In [3]:
TARGET_PRICE = 10000

In [4]:
CHECK_INTERVAL = 300

In [5]:
headers = {
    'authority': 'www.amazon.com',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    "User-Agent1": 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:138.0) Gecko/20100101 Firefox/138.0',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [6]:
# Function to fetch the webpage
def get_amazon_page():
    time.sleep(2)  # Small delay to avoid being blocked
    page = requests.get(url, headers=headers)
    print(f"Status Code: {page.status_code}")
    print(f"URL: {page.url}")
    return page

In [7]:
# Function to extract product title
def get_title(soup):
    try:
        title = soup.find_all('h1', {'id': 'title'})[0].find_all('span')[0].text.strip()
        return title
    except:
        try:
            title = soup.find('span', {'id': 'productTitle'}).text.strip()
            return title
        except:
            return "Unknown Product"


In [8]:
def get_price(soup):
    try:
        price_text = soup.find_all('span', {'class': 'a-price-whole'})[0].text.strip()
        price_text = price_text.replace(',', '').replace('.', '')
        price = int(price_text)
        return price
    except Exception as e:
        print(e)
        return None

In [9]:
def amazon_send_email(title, price):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()

    server.login('divyansh.raghav@contify.com', 'bimymxkesiikjcey')

    subject = f"Price Alert: {title} is now Rs.{price}"
    body = f"The price of {title} has dropped to Rs.{price}!\n"

    message = f"Subject: {subject}\n\n{body}"

    server.sendmail(
        'divyansh.raghav@contify.com',
        'raghavdivyansh9@gmail.com',
        message
    )

    server.quit()
    print(f"Email sent at {datetime.datetime.now()}")

In [10]:
def check_price_once():
    page = get_amazon_page()
    soup = BeautifulSoup(page.content, 'html.parser')

    title = get_title(soup)
    price = get_price(soup)

    print(f"Product: {title}")
    print(f"Current Price: ₹{price}")

    return title, price

In [11]:
title, price = check_price_once()

Status Code: 200
URL: https://www.amazon.in/Redmi-A4-5G-Sparkle-Charging/dp/B0DLW427YG/ref=sr_1_1_sspa?_encoding=UTF8&content-id=amzn1.sym.bd428427-e2fa-455b-845a-a54edb5f536f&dib=eyJ2IjoiMSJ9.Zu16M0f4gcJTITvmFzGqs09EXESHBn36UX9vRwLtp43PhS-3_IY2Fs73Ku6LbRlMIJ-BGLDG7ZZd2ctqJbqxv7vk6mrTF2l87iQWhN2qtRYWXCRutAfVXjCa6s9k8lYTdcgifhH_faDWuAyM1EmFgkLcz5HCH5PoTuKiJkZjPjGH1xy1OtH05DxdSO3P-dJ1heNXxYWzobMdKRP3M5E4Ta0jb0TVF9ZnoKSzGwgWY6u7GVXjITUOJHg7UoPtX3GxEtJrG6wAvgL4S38ajygckC1WHO4hX_D2y3HIHFLe8Lc.L5Y8AffFc-O4ioP4KMYOvwBicFzKvGvZrsYAoA_GYXU&dib_tag=se&pd_rd_r=56178e86-1b26-4ccb-a9fe-28a25be7649f&pd_rd_w=A11pD&pd_rd_wg=MYv29&qid=1746590515&refinements=p_36%3A-1030000&rnid=1318502031&s=electronics&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&th=1
Product: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Current Price: ₹7999


In [12]:
def track_price_continuously():
    print(f"Starting price tracker. Target price: ₹{TARGET_PRICE}")
    print(f"Checking every {CHECK_INTERVAL} seconds")

    last_price = None
    notification_sent = False

    try:
        while True:
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"\nChecking price at {current_time}")

            title, price = check_price_once()

            if price is None:
                print("Could not retrieve price, will try again next interval")
            else:
                # Check if price has changed
                if last_price is not None and price != last_price:
                    print(f"Price changed from Rs.{last_price} to Rs.{price}")

                # Check if price is below target and notification hasn't been sent yet
                if price <= TARGET_PRICE and not notification_sent:
                    print(f"Price Rs.{price} is below target Rs.{TARGET_PRICE}! Sending notification...")
                    amazon_send_email(title, price)
                    notification_sent = True

                # Reset notification flag if price goes above target again
                elif price > TARGET_PRICE and notification_sent:
                    print("Price is above target again. Will send another notification if it drops.")
                    notification_sent = False

                last_price = price

            print(f"Next check in {CHECK_INTERVAL} seconds")
            time.sleep(CHECK_INTERVAL)

    except KeyboardInterrupt:
        print("\nPrice tracking stopped by user")
    except Exception as e:
        print(f"An error occurred: {e}")

In [13]:
title, price = check_price_once()

Status Code: 200
URL: https://www.amazon.in/Redmi-A4-5G-Sparkle-Charging/dp/B0DLW427YG/ref=sr_1_1_sspa?_encoding=UTF8&content-id=amzn1.sym.bd428427-e2fa-455b-845a-a54edb5f536f&dib=eyJ2IjoiMSJ9.Zu16M0f4gcJTITvmFzGqs09EXESHBn36UX9vRwLtp43PhS-3_IY2Fs73Ku6LbRlMIJ-BGLDG7ZZd2ctqJbqxv7vk6mrTF2l87iQWhN2qtRYWXCRutAfVXjCa6s9k8lYTdcgifhH_faDWuAyM1EmFgkLcz5HCH5PoTuKiJkZjPjGH1xy1OtH05DxdSO3P-dJ1heNXxYWzobMdKRP3M5E4Ta0jb0TVF9ZnoKSzGwgWY6u7GVXjITUOJHg7UoPtX3GxEtJrG6wAvgL4S38ajygckC1WHO4hX_D2y3HIHFLe8Lc.L5Y8AffFc-O4ioP4KMYOvwBicFzKvGvZrsYAoA_GYXU&dib_tag=se&pd_rd_r=56178e86-1b26-4ccb-a9fe-28a25be7649f&pd_rd_w=A11pD&pd_rd_wg=MYv29&qid=1746590515&refinements=p_36%3A-1030000&rnid=1318502031&s=electronics&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&th=1
Product: Redmi A4 5G (Sparkle Purple, 4GB RAM, 64GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Current Price: ₹7999


In [14]:
# track_price_continuously()

In [15]:
amazon_send_email(title, price)





























































































































































































































































































KeyboardInterrupt: 